In [1]:
import pandas
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
data3 = np.genfromtxt('data3.dat', delimiter='  ')
data3 = data3.transpose()
drawsml = np.genfromtxt('drawsml.dat', delimiter='  ')
drawsgmm = np.genfromtxt('drawsgmm.dat', delimiter='  ')

In [18]:
def sim_ll(y,x1,x2,t1,t2,sigma,v=False):
    #hard coded draws
    eps1 = np.genfromtxt('drawsml.dat', delimiter='  ').transpose()
    sim_like = 0
    
    #set up vectors for adding
    zeros = np.full(len(x1),.0001)
    const = np.full(len(x1), 100)
    
    #run simulations, to simulate likelihoods
    for s in range(len(eps1)):
        
        #calculate f inverves
        mc1 = np.exp(t1 + t2*x1 + sigma*eps1[s])
        safe_log1 = np.maximum( 3*y - const - mc1 ,zeros)
        f_inv =  (np.log(safe_log1) - t1 - t2*x2)
        safe_f_inv = np.maximum( f_inv ,zeros)
        #calculate f prime
        f_prime = (3.0/sigma)*1/(3.0*y - const - mc1)
        
        sim_like = sim_like + norm(0,sigma).pdf(f_inv)*f_prime
    
    #finalized simulated likelihoods
    safe_sim = np.maximum(sim_like/len(eps1),zeros)    
    return np.log(safe_sim).sum()


class part_b(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        t1, t2, sigma = params
        y = self.endog
        x1,x2 = self.exog.transpose()
        return -sim_ll(y, x1,x2,t1,t2,sigma)

    
    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwds):
        # we have one additional parameter and we need to add it for summary
        self.exog_names.append('sigma')
        if start_params == None:
            # Reasonable starting values
            start_params = np.append(np.zeros(self.exog.shape[1]), [.5])
        return super(part_b, self).fit(start_params=start_params, maxiter=100, maxfun=maxfun, **kwds)

    
model_b = part_b(data3[0],data3[1:].transpose()).fit(start_params = [1,1,1])
print(model_b.summary())

Optimization terminated successfully.
         Current function value: -3.174360
         Iterations: 92
         Function evaluations: 166
                                part_b Results                                
Dep. Variable:                      y   Log-Likelihood:                 158.72
Model:                         part_b   AIC:                            -313.4
Method:            Maximum Likelihood   BIC:                            -309.6
Date:                Mon, 24 Sep 2018                                         
Time:                        17:06:23                                         
No. Observations:                  50                                         
Df Residuals:                      48                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

In [25]:
def sim_g(t1,t2,sigma,y,x1,x2,eps1,eps2):
    g = 0
    for s in range(len(eps1)):
        #this expression may be wrong
        y_hat = 1/3.0*(100 + np.exp(t1 + t2*x1 + sigma*eps1[s]) + np.exp(t1 + t2*x2 + sigma*eps2[s]) )
        g = g + (y - y_hat)
        
    return g/ (len(eps1)*len(eps2))


def sim_g_sq(t1,t2,sigma,y,x1,x2,eps1,eps2):
    g = 0
    for s in range(len(eps1)):
        #this expression may be wrong
        y_hat = 1/3.0*(100 + np.exp(t1 + t2*x1 + sigma*eps1[s]) + np.exp(t1 + t2*x2 + sigma*eps2[s]) )
        g = g + (y**2-y_hat**2)
    return g/(len(eps1)*len(eps2))


class part_c(GMM):
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        kwds.setdefault('k_moms', 4)
        kwds.setdefault('k_params',3)
        super(part_c, self).__init__(*args, **kwds)

    def momcond(self, params):
        t1,t2, sigma = params   
        y = self.endog
        x1,x2 = self.exog.transpose()
        
        #initialize draws
        eps = np.genfromtxt('drawsgmm.dat', delimiter='  ').transpose()
        eps1 = eps[0:len(eps)/2]
        eps2 = eps[len(eps)/2:len(eps)]
        #calc moment conditions
        g = sim_g(t1,t2,sigma,y,x1,x2,eps1,eps2)
        g_sq = sim_g_sq(t1,t2,sigma,y,x1,x2,eps1,eps2)
        return np.column_stack((g,g*x1,g*x2,g_sq))


model_c = part_c(data3[0],data3[1:].transpose(), None)
model_c.set_param_names(['theta_1','theta_2','sigma'])
beta0 = np.array([1, 1,1])
resc = model_c.fit(beta0, maxiter=3, optim_method='nm', wargs=dict(centered=False))
print(resc.summary())

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 42
         Function evaluations: 84
Optimization terminated successfully.
         Current function value: 0.004433
         Iterations: 73
         Function evaluations: 136
Optimization terminated successfully.
         Current function value: 0.015908
         Iterations: 55
         Function evaluations: 104
                                part_c Results                                
Dep. Variable:                      y   Hansen J:                       0.7954
Model:                         part_c   Prob (Hansen J):                 0.372
Method:                           GMM                                         
Date:                Thu, 20 Sep 2018                                         
Time:                        15:35:52                                         
No. Observations:                  50                                         
                 coef    std er